# Flip-contrained SCA

For SCA dynamics, we plot a sample of the modified Hamiltonian in "Step-Energy graph".  For flip-constrained SCA dynamics, we plot a sample of the (original) Hamiltonian in "Step-Energy graph".

## Annealing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
#import sys
#sys.path.append('../python')
#import simulator
import simulatorWithCpp as simulator
import math

%matplotlib inline
np.set_printoptions(threshold=16, edgeitems=8)

In [ ]:
MaxSteps = int(1.e3)
MaxTrials = int(1.e2)
NumNodes = 64
SeedForConfiguration = 1024

In [ ]:
isingModel = simulator.IsingModel({node: 0.e0 for node in range(NumNodes)}, {})
isingModel.SetSeed(SeedForConfiguration)
isingModel.GiveSpins(simulator.ConfigurationsType.Uniform)
InitialConfiguration = isingModel.Spins

def TryExperimentFor(isingModel, initialTemperature):
    minimumEnergiesData = np.empty(0, dtype=np.float)
    samples = np.empty((MaxSteps + 1, 4), dtype=np.float)
    #isingModel.Write()
    for i in range(MaxTrials):
        isingModel.Spins = InitialConfiguration
        isingModel.SetSeed()
        for n in range(MaxSteps + 1):
            isingModel.Temperature = initialTemperature * 0.99 ** n
            #isingModel.Temperature = 100 * np.exp(-0.005 * n)
            isingModel.Update()
            samples[n, 0] = n
            samples[n, 1] = isingModel.Temperature
            samples[n, 2] = isingModel.Energy
            samples[n, 3] = isingModel.EnergyOnBipartiteGraph
        minimumEnergiesData = np.append(minimumEnergiesData, samples[:, 2].min())

    print('Mean: {}'.format(np.mean(minimumEnergiesData)))
    print('Standard deviation: {}'.format(np.std(minimumEnergiesData)))
    print('Mode: {}'.format(stats.mode(minimumEnergiesData)))
    print('Minimum: {}'.format(np.min(minimumEnergiesData)))

    fig = plt.figure(figsize=(7, 3), dpi=200)
    ax = fig.add_subplot(121, xlabel='MC steps', ylabel='Energy')
    ax.grid()
    ax.plot(samples[:, 0], samples[:, 2], label='Original Hamiltonian')
    ax.plot(samples[:, 0], samples[:, 3], label='Double Hamiltonian')
    ax.legend()
    ax = fig.add_subplot(122, xlabel='Energy', ylabel='Frequency')
    ax.grid(which='both')
    ax.hist(minimumEnergiesData, bins=30)
    fig.suptitle(isingModel.Algorithm.name)
    plt.subplots_adjust(wspace=0.3)
    plt.show()

In [ ]:
def GenerateSquareLatticeEdges(numNodes):
    columns = math.ceil(math.sqrt(numNodes))
    result = {}
    for i in range(numNodes - 1):
        if (i + 1) % columns > 0:
            result[(i, i + 1)] = -1
        if (i + columns) < numNodes:
            result[(i, i + columns)] = -1
    return result

quadratic = GenerateSquareLatticeEdges(NumNodes)
isingModel = simulator.IsingModel({}, quadratic)
T0 = 2.e0 * np.sum([np.abs(J) for J in quadratic.values()])

In [ ]:
isingModel.Algorithm = simulator.Algorithms.SCA
isingModel.PinningParameter = 0.5e0 * isingModel.CalcLargestEigenvalue()
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

In [ ]:
isingModel.Algorithm = simulator.Algorithms.fcSCA
isingModel.PinningParameter = 0.125e0 * isingModel.CalcLargestEigenvalue()
isingModel.FlipTrialRate = 0.8e0
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

In [ ]:
def GenerateCompleteGraphEdges(numNodes):
    return {(i, j): -1 for i in range(numNodes) for j in range(i + 1, numNodes)}

quadratic = GenerateCompleteGraphEdges(NumNodes)
isingModel = simulator.IsingModel({}, quadratic)
T0 = 2.e0 * np.sum([np.abs(J) for J in quadratic.values()])

In [ ]:
isingModel.Algorithm = simulator.Algorithms.SCA
isingModel.PinningParameter = 0.5e0 * isingModel.CalcLargestEigenvalue()
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

In [ ]:
isingModel.Algorithm = simulator.Algorithms.fcSCA
isingModel.PinningParameter = 0.125e0 * isingModel.CalcLargestEigenvalue()
isingModel.FlipTrialRate = 0.5e0
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

In [ ]:
OccupationProbability = 0.5e0
SeedForRandomGraph = 2048

def GenerateErdosRenyiEdges(numNodes, probability):
    rng = np.random.Generator(np.random.MT19937(SeedForRandomGraph))
    return {(i, j): -1 if rng.random() <= probability else 0 for i in range(numNodes) for j in range(i + 1, numNodes)}

quadratic = GenerateErdosRenyiEdges(NumNodes, OccupationProbability)
isingModel = simulator.IsingModel({}, quadratic)
T0 = 2.e0 * np.sum([np.abs(J) for J in quadratic.values()])

In [ ]:
isingModel.Algorithm = simulator.Algorithms.SCA
isingModel.PinningParameter = 0.5e0 * isingModel.CalcLargestEigenvalue()
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

In [ ]:
isingModel.Algorithm = simulator.Algorithms.fcSCA
isingModel.PinningParameter = 0.125e0 * isingModel.CalcLargestEigenvalue()
isingModel.FlipTrialRate = 0.2e0
TryExperimentFor(isingModel, T0 + NumNodes * isingModel.PinningParameter)

## Stationary distribution

In [ ]:
NumNodes = 10

isingModel = simulator.IsingModel({}, GenerateCompleteGraphEdges(NumNodes))
isingModel.Algorithm = simulator.Algorithms.fcSCA
isingModel.PinningParameter = 0.e0
isingModel.FlipTrialRate = 0.8e0
isingModel.Temperature = NumNodes ** 2
isingModel.SetSeed(SeedForConfiguration)
isingModel.GiveSpins(simulator.ConfigurationsType.Uniform)

In [ ]:
MaxSteps = 2 ** (2 * NumNodes)

samples = np.empty((MaxSteps + 1, 2), dtype=np.float)
for i in range(MaxSteps + 1):
    isingModel.Update()
    #samples[n, 0] = 
    #samples[n, 1] = isingModel.Energy

In [ ]:
for spin in isingModel.Spins.values():
    print(spin)